In this notebook, we train a model to learn a strategy for Blackjack using a decision tree regression.

Below we write an `actionpicker` function that decides what `action` to play given the `observation` from the Blackjack simulator, `env`.
`epsilon` percent of the time, we decide on a random `action`.

We also write a `train` function fits our decision tree regression to our `dat` and `labs`.

In [ ]:
def actionpicker(clf, env, observation, epsilon):
  if clf == 0:
    action = env.action_space.sample()
  else:
    pred = clf.predict([observation + (0,), observation + (1,)])
    action = 1*(pred[1]>pred[0])
  if random.random() < epsilon:
    action = env.action_space.sample()
  return(action)

def train(dat, lab):
  clf = tree.DecisionTreeRegressor(max_leaf_nodes = 6)	
  clf = clf.fit(dat, lab)
  return(clf)

We will be using Blackjack environment provided by gym API.
It is a simplified Blackjack, where the player can only hit or stay at each turn.
Other actions, such as split or double down, are not supported.
Each card draw is from a fresh deck -- the API doesn't keep track of which cards have already been drawn.

To create a new Blackjack environment:
```
env = gym.make("Blackjack-v0", natural=False)
```
The optional parameter `natural` (which must be a keyword parameter instead of positional) determines whether 1.5 time reward is given to a user blackjack. By default no extra reward is given to blackjack.

The Blackjack environment has three primary methods:
- `env.reset() -> tuple[int, int, bool]` starts a new round, dealing two cards to the user and two cards to the dealer.
  It returns a 3-tuple: total value of user's cards (with Ace valued at 11 if possible),
  value of dealer's first card (dealer's second card is hidden at this point),
  and whether the user has an Ace that is currently used as 11.
  For the value of dealer's first card, Ace is represented by 1, regardless of whether it should be used as 11 or not.
- `env.action_space.sample() -> int` randomly selects an action for the user.
  It returns 0 (meaning stay) or 1 (meaning hit) with 50% probability each.
- `env.step(action: int) -> tuple[tuple[int, int, bool], float, bool, dict]` takes one user action (either 0 for stay, or 1 for hit) and completes it.
  Returns a 4-tuple. Item 0 is observation, a 3-tuple with the same meaning as that returned by `env.reset`;
  item 1 is reward, amount won by the user; item 2 is done, indicating whether this round is done or not;
  item 4 is an empty dict, probably for future use. `reward` is 0.0 if `done` is False.
  - When `action` is 1, the code draws one card for the user. If the user is bust, the round completes and no action is possible until the env resets. Otherwise, the round is not complete, and the user can make another action.
  - When `action` is 0, the code draws cards for dealer until the total value reaches at least 17, and the end round is complete.

There are two attributes, `env.player` and `env.dealer`, that are useful for debugging. The return a list of all the cards currently held by the player and the dealer, respectively. Note that in a real game, the player can't see `env.dealer[1:]` while makeing a decision.

The code below also calls `env.close()`, which actually does nothing.

In the next code block, we train our Blackjack playing model and visualize the descision tree regression we have learned.  We will train our model 5 times, playing 100,000 rounds, and print the number of rounds it won each time.

In [ ]:
# action 0 is stay
# action 1 is hit

import gym
from sklearn import tree
import random

env = gym.make("Blackjack-v0")
epochs = 5
N = 100000
epsilon = 0.1
clf = 0

# In epoch 0, the player always chooses random actions.
# At the end of each epoch, a decision tree classifier is trained using the data
# from this epoch. The classifier is then used in the next epoch to choose actions.
for epoch in range(epochs):
  dat = []
  lab = []
  wins = 0
  for _ in range(N):
    done = False
    observation = env.reset()
    while not done:
      # pick an action
      action = actionpicker(clf, env, observation, epsilon)

      dat += [observation + (action,)]
      observation, reward, done, info = env.step(action)
      if done:
        target = reward
      elif epoch == 0:
        target = 0
      else:
        pred = clf.predict([observation + (0,), observation + (1,)])
        target = max(pred)
      lab += [target]
      if reward > 0.0: wins += 1
  clf = train(dat, lab)

  env.close()
#  print(dat)
  print('Epoch %d: %d wins (%.2f%%)' % (epoch, wins, wins/N*100))
  print('New classifier:')
  print(tree.export_text(clf, class_names = [-1, 0, 1],
                         feature_names = ["holding", "dealer", "ace", "action"]))
  print()

After repeated runs of the above code cell, the results always look like this:
- Epoch 0 wins about 28% of the rounds.
- Epoch 1 wins about 41% of the rounds.
- Epoch 2 through 4 each wins about 37.5%.

Interestingly, the best classifier is created from records of random actions (created at the end of epoch 0 and used in epoch 1). But even the best classifier has less than 50% wins. The code doesn't record the count of ties vs losses, so we don't know the actual win/loss amount in each round. value: [-0.51] value: [-0.58]value: [0.00]



The classifier generated at the end of epoch 0 generally looks like:
```
|--- action <= 0.50
|   |--- holding <= 18.50
|   |   |--- value: [-0.38]
|   |--- holding >  18.50
|   |   |--- value: [0.57]
|--- action >  0.50
|   |--- holding <= 13.50
|   |   |--- value: [-0.16]
|   |--- holding >  13.50
|   |   |--- ace <= 0.50
|   |   |   |--- holding <= 17.50
|   |   |   |   |--- value: [-0.57]
|   |   |   |--- holding >  17.50
|   |   |   |   |--- value: [-0.88]
|   |   |--- ace >  0.50
|   |   |   |--- value: [0.00]
```

The classifier generated at the end of epoch 1 generally looks like:
```
|--- holding <= 18.50
|   |--- holding <= 11.50
|   |   |--- value: [-0.06]
|   |--- holding >  11.50
|   |   |--- holding <= 17.50
|   |   |   |--- value: [-0.36]
|   |   |--- holding >  17.50
|   |   |   |--- value: [-0.05]
|--- holding >  18.50
|   |--- action <= 0.50
|   |   |--- holding <= 19.50
|   |   |   |--- value: [0.27]
|   |   |--- holding >  19.50
|   |   |   |--- value: [0.69]
|   |--- action >  0.50
|   |   |--- value: [-0.58]
```

The classifier generated at the end of epoch 2 or later generally looks like:
```
|--- holding <= 18.50
|   |--- dealer <= 7.50
|   |   |--- dealer <= 1.50
|   |   |   |--- value: [-0.71]
|   |   |--- dealer >  1.50
|   |   |   |--- value: [-0.20]
|   |--- dealer >  7.50
|   |   |--- value: [-0.50]
|--- holding >  18.50
|   |--- action <= 0.50
|   |   |--- holding <= 19.50
|   |   |   |--- value: [0.26]
|   |   |--- holding >  19.50
|   |   |   |--- value: [0.67]
|   |--- action >  0.50
|   |   |--- value: [-0.51]
```

Each run of the code will likely generate slightly different values for each leaf node, but the splitting rules at each internal node are generally very consistent. These decision trees may look odd to human readers:
- In the tree for epoch 0, the action splits at the root of the tree.
- In the trees for epoch 1, 2, or later, the branch for holding <= 18.50 has no mention of action, so this branch is useless for making a decision on the action.